In [4]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam

# Data loading
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Model building
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Save the model
model.save('skin_disease_detection_model.h5')


Found 697 images belonging to 9 classes.


Found 181 images belonging to 9 classes.
9406464/9406464 [==============================] - 19s 2us/step


Epoch 1/10


2024-03-08 04:25:16.629990: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154140672 exceeds 10% of free system memory.
2024-03-08 04:25:16.840089: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154140672 exceeds 10% of free system memory.
2024-03-08 04:25:19.159233: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 156905472 exceeds 10% of free system memory.
2024-03-08 04:25:21.243596: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57802752 exceeds 10% of free system memory.
2024-03-08 04:25:21.304530: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57802752 exceeds 10% of free system memory.


21/21 [==============================] - 234s 9s/step - loss: 1.2952 - accuracy: 0.5684 - val_loss: 14.6723 - val_accuracy: 0.1187
Epoch 2/10
21/21 [==============================] - 149s 7s/step - loss: 0.6823 - accuracy: 0.7654 - val_loss: 12.6973 - val_accuracy: 0.1250
Epoch 3/10
21/21 [==============================] - 176s 8s/step - loss: 0.4148 - accuracy: 0.8662 - val_loss: 21.4724 - val_accuracy: 0.1000
Epoch 4/10
21/21 [==============================] - 212s 10s/step - loss: 0.3882 - accuracy: 0.8602 - val_loss: 20.3269 - val_accuracy: 0.1688
Epoch 5/10
21/21 [==============================] - 188s 9s/step - loss: 0.3607 - accuracy: 0.8872 - val_loss: 23.0654 - val_accuracy: 0.1187
Epoch 6/10
21/21 [==============================] - 160s 8s/step - loss: 0.2424 - accuracy: 0.9083 - val_loss: 22.0756 - val_accuracy: 0.1437
Epoch 7/10
21/21 [==============================] - 155s 7s/step - loss: 0.2488 - accuracy: 0.9293 - val_loss: 12.8833 - val_accuracy: 0.1625
Epoch 8/10
21/21

/home/dragon/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Data loading
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 256 hidden units and ReLU activation
x = Dense(256, activation='relu')(x)

# Add a final classification layer with softmax activation
predictions = Dense(9, activation='softmax')(x)

# Combine the base model and the new classification layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model (only train the new classification layers)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f'Validation loss: {loss:.4f}')
print(f'Validation accuracy: {accuracy:.4f}')


Found 697 images belonging to 9 classes.
Found 181 images belonging to 9 classes.
94765736/94765736 [==============================] - 174s 2us/step


Epoch 1/10
21/21 [==============================] - 422s 17s/step - loss: 2.3400 - accuracy: 0.0977 - val_loss: 2.2529 - val_accuracy: 0.1063
Epoch 2/10
21/21 [==============================] - 316s 15s/step - loss: 2.2680 - accuracy: 0.0977 - val_loss: 2.2278 - val_accuracy: 0.1187
Epoch 3/10
21/21 [==============================] - 234s 11s/step - loss: 2.2456 - accuracy: 0.1173 - val_loss: 2.2197 - val_accuracy: 0.1125
Epoch 4/10
21/21 [==============================] - 168s 8s/step - loss: 2.2301 - accuracy: 0.1038 - val_loss: 2.2209 - val_accuracy: 0.1937
Epoch 5/10
21/21 [==============================] - 164s 8s/step - loss: 2.2355 - accuracy: 0.1203 - val_loss: 2.2414 - val_accuracy: 0.1187
Epoch 6/10
21/21 [==============================] - 185s 9s/step - loss: 2.2259 - accuracy: 0.1248 - val_loss: 2.2239 - val_accuracy: 0.1750
Epoch 7/10
21/21 [==============================] - 158s 8s/step - loss: 2.2137 - accuracy: 0.1278 - val_loss: 2.1964 - val_accuracy: 0.1250
Epoch 8/10

In [4]:
import os
import numpy as np
from PIL import Image
from flask import Flask, request, jsonify, render_template
from tensorflow import keras

app = Flask(__name__)
model = keras.models.load_model('skin_disease_detection_model.h5')
target_size = (224, 224)  # Adjust based on your model's input shape

def preprocess_image(image):
    image = image.resize(target_size)
    image = np.array(image) / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def predict_image(image):
    preprocessed_image = preprocess_image(image)
    predictions = model.predict(preprocessed_image)
    predicted_class = np.argmax(predictions, axis=-1)
    return predicted_class

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        image = Image.open(file)
        predicted_class = predict_image(image)
        return render_template('index.html', predicted_class=int(predicted_class))

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5033
Press CTRL+C to quit
 * Restarting with stat
0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "/home/dragon/.local/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/dragon/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/dragon/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dragon/.local/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    

SystemExit: 1

In [5]:
# Get the current working directory
cwd = os.getcwd()

# Construct the path to the model file
model_path = os.path.join(cwd, 'skin_disease_detection_model.h5')

# Load the model
model = keras.models.load_model(model_path)
model_path

'/home/dragon/repos/MachineLEarning-School/face detection/skin_disease_detection_model.h5'

: 